# Trabajo práctico N°2 - Regresión polinómica.

# Exploración de datos.

In [145]:
#librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### Se definen los conjuntos de entrenamiento, testeo y validación.

In [146]:

#se cargan los datos del csv
df = pd.read_csv('car_data.csv')

#Se eliminan del df las columnas del nombre del auto y su dueño.
df.drop('Car_Name', axis=1, inplace=True)  #Se modifica el df en sí, no genera una copia.
df.drop('Owner', axis=1, inplace=True) 

#Se definen los conjuntos de entrenamiento, testeo y validación
train = df[df['Set'] == 'Train'].copy()
test = df[df['Set'] == 'Test'].copy()
validation = df[df['Set'] == 'Validation'].copy()

#Se elimina la columna Set del df original
df.drop('Set', axis=1, inplace=True) 


##### Se calculan las proporciones de las variables categóricas, representando las probabilidades de cada una.

In [147]:
#Se calculan las probabilidades para los tipos de combustible (Fuel_Type)
prob_fuel_type = df['Fuel_Type'].value_counts(normalize=True)

#Se calculan las probabildiades para lso tipos de vendedores (Seller_Type)
prob_seller_type = df['Seller_Type'].value_counts(normalize=True)

#Se calculan las probabilidades de los tipos de transmisiones (Transmission)
prob_transmission_type = df['Transmission'].value_counts(normalize=True)

print(prob_fuel_type,'\n')
print(prob_seller_type,'\n')
print(prob_transmission_type,'\n')

Fuel_Type
Petrol    0.794020
Diesel    0.199336
CNG       0.006645
Name: proportion, dtype: float64 

Seller_Type
Dealer        0.647841
Individual    0.352159
Name: proportion, dtype: float64 

Transmission
Manual       0.86711
Automatic    0.13289
Name: proportion, dtype: float64 



# Pre-procesamiento.

In [148]:
#librerias
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

##### Codificación.

In [149]:
#Se codifican las variables categóricas

#Variables binarias - Se definen los menos probables como True - con el fin de aprovechar el Sparse
# Transmission: Automatic está definido True (pasado a 1 por .astype(int))
transmission_encoded = (df['Transmission'] == 'Automatic').astype(int)

# Seller_Type: Individual está definido como True (pasado a 1 por .astype(int))
seller_encoded = (df['Seller_Type'] == 'Individual').astype(int)
 
#Variables de 3 categorías:
 #Se crea el encoder como un array denso (no elimina los 0's)
fuel_encoder = OneHotEncoder(sparse_output=False)

 #Se transforma la columna del df a una codificación one-hot, utilizando el fuel_encoder
fuel_encoded = fuel_encoder.fit_transform(df[['Fuel_Type']])

 #Se aprecia el orden de las categorias de combustible ya codificadas
print(fuel_encoder.categories_)


[array(['CNG', 'Diesel', 'Petrol'], dtype=object)]


##### Mapa polinómico.

In [150]:
#Se seleccionan las columnas con valores exceptuando las categóricas y el precio de venta.
values = df.drop(columns = ['Selling_Price','Transmission','Seller_Type','Fuel_Type' ]).copy()

#Se crea el mapa polinómico de grado 3, sin incluir la columna de 1's
poly = PolynomialFeatures(degree=3, include_bias=False)
values_poly = poly.fit_transform(values)

#Se concatenan todos los datos en un array de numpy antes de pasarlos a ColumnTransformer
data_combined = np.c_[
    values_poly,
    transmission_encoded.values,
    seller_encoded.values,
    fuel_encoded
]

#Se prepara el transformador de los datos, indicando la normalización
preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), list(range(data_combined.shape[1])))
])


# Regresión Lineal.

In [151]:
#Se define el pipeline, indicando primero la normalización y luego el entrenamiento de la regresión lineal
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('regressor', LinearRegression())
])

#Se define la variable a predecir
y = df['Selling_Price'].values

#Se entrena el modelo, utilizando los datos concatenados, con el objetivo de predecir y
pipeline.fit(data_combined, y)

#Se realiza la predicción de y, utilizando los datos entrenados
y_pred = pipeline.predict(data_combined)

# Regularización.